In [1]:
import numpy as np
import pandas as pd
import healpy as hp

In [2]:
filepath = '/Users/jimenagonzalez/research/DSPL/Hybrid_SL_Modeling/data/candidates_tocompare/ODonnell/'

#Reading table3 - Properties of candidate lensing systems
colspecs = [(0,3), (3,14), (15,17), (18,47), (48,53), (54,58), (59,85), (86, 96)]
table3 = pd.read_fwf( filepath + "table3.dat", colspecs=colspecs,
    names=["Prefix", "DES", "Rank", "Algorithm", "Rad", "e_Rad", "OName", "Ref"])
table3.head()

,Prefix,DES,Rank,Algorithm,Rad,e_Rad,OName,Ref
0,DES,J0002-3332,4,BNA,5.69,0.83,NaN,NaN
1,DES,J0006-4208,10,ACT,7.70,0.81,NaN,B
2,DES,J0007-4434,5,RedM,3.43,0.43,NaN,"B, D"
3,DES,J0011-4614,7,"ACT, RedM",6.53,2.35,NaN,"B, D"
4,DES,J0014-0057,5,"ACT, BNA, RedM missed",12.84,1.31,SOGRAS J0014-0057,"h, w"


In [3]:
#Reading table4 - *Names, positions, photometry, and photometric redshifts of objects for each candidate system
colspecs = [(0,3), (3,14), (17, 18), (18, 19), (19, 20), (21,31), (32,42), (43, 44), (45,50), (51,56), (57,62), (63,68), (69,74), (75,80),
(81,86), (87,91), (92,97), (98,103), (104,107), (107,112), (113,119)]
table4 = pd.read_fwf(filepath + "table4.dat", colspecs=colspecs,
    names=["Prefix", "DES", "Paren1", "m_DES", "Paren2", "RAdeg", "DEdeg", "f_m_DES", "gmag", "e_gmag", "rmag", "e_rmag",
    "imag", "e_imag", "zmag", "e_zmag", "Ymag", "e_Ymag", "Flag", "zphot", "e_zphot"])
table4 = table4.drop(columns=["Paren1", "Paren2"])


In [4]:
print(len(table3), len(table4))
print(len(table4[table4['m_DES'] == 'A']))

table4_lenses = table4[(table4['m_DES'] == 'A') | (table4['m_DES'] == 'B')] #Getting only the deflector, not the source images
table4_lenses.head()

252 652
236


,Prefix,DES,m_DES,RAdeg,DEdeg,f_m_DES,gmag,e_gmag,rmag,e_rmag,imag,e_imag,zmag,e_zmag,Ymag,e_Ymag,Flag,zphot,e_zphot
0,DES,J0002-3332,A,0.527213,-33.544062,NaN,22.67,0.08,21.00,0.02,20.35,0.02,20.02,0.04,19.92,0.08,p,0.50,0.03
1,DES,J0002-3332,B,0.527307,-33.542925,NaN,22.45,0.05,21.12,0.02,20.70,0.02,20.45,0.04,20.44,0.11,p,0.48,0.08
4,DES,J0007-4434,A,1.872055,-44.579515,NaN,20.42,0.01,18.73,0.00,18.01,0.00,17.62,0.01,17.45,0.01,p,0.51,0.01
6,DES,J0011-4614,A,2.971390,-46.239450,NaN,20.18,0.01,19.14,0.00,18.46,0.00,18.00,0.01,17.84,0.01,*,0.57,0.02
9,DES,J0014-0057,A,3.725464,-0.952317,NaN,21.14,0.03,19.24,0.01,18.45,0.01,18.04,0.01,17.94,0.02,p,0.53,0.00


In [5]:
new_table = table3.merge(table4_lenses[['RAdeg', 'DEdeg', 'DES']], on = 'DES')
new_table['Name'] = new_table['Prefix'] + ' ' + new_table['DES']
new_table['Rank_norm'] = 3 * (new_table['Rank'] - new_table['Rank'].min()) / (new_table['Rank'].max() - new_table['Rank'].min())
new_table = new_table.rename({'Rad': 'AverRad', 'e_Rad': 'e_AverRad', 'RAdeg': 'RA', 'DEdeg': 'DEC', 'Rank_norm': 'Exp_score'}, axis=1)
new_table = new_table[['Name', 'RA', 'DEC', 'Exp_score', 'AverRad', 'e_AverRad']]

#Adding healpix
ra_rad, dec_rad = np.deg2rad(new_table['RA'].values), np.deg2rad(new_table['DEC'].values)
theta, phi = 0.5 * np.pi - dec_rad, ra_rad 
new_table['HPIX_4096'] = hp.ang2pix(4096, theta, phi, nest=False)

new_table.to_csv('Initial_ODonnell.csv', index=False)
new_table.head()

,Name,RA,DEC,Exp_score,AverRad,e_AverRad,HPIX_4096
0,DES J0002-3332,0.527213,-33.544062,0.428571,5.69,0.83,156278808
1,DES J0002-3332,0.527307,-33.542925,0.428571,5.69,0.83,156278808
2,DES J0007-4434,1.872055,-44.579515,0.857143,3.43,0.43,171318668
3,DES J0011-4614,2.971390,-46.239450,1.714286,6.53,2.35,173358995
4,DES J0014-0057,3.725464,-0.952317,0.857143,12.84,1.31,102326441
